In [70]:
import warnings
warnings.filterwarnings('ignore')

In [71]:
from numpy.random import seed
seed(42)

In [72]:
import os
import pandas as pd
import math
import numpy as np

In [73]:
! pip install wget


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [74]:
! pip install holidays


[notice] A new release of pip available: 22.1.2 -> 22.2.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [75]:
import wget
import os.path

if(not os.path.exists('finale.csv')):
    wget.download('https://data.4tu.nl/ndownloader/files/23993303')

In [76]:
df_finale = pd.read_csv('finale.csv', index_col=0).reset_index()

In [77]:
df_finale.columns

Index(['Case ID', 'Activity', 'Resource', 'Complete Timestamp', 'Variant',
       'Variant index', 'Variant.1', 'seriousness', 'customer', 'product',
       'responsible_section', 'seriousness_2', 'service_level', 'service_type',
       'support_section', 'workgroup'],
      dtype='object')

In [78]:
df_finale.shape

(21348, 16)

String to datetime conversion

In [79]:
df_finale['Complete Timestamp'] = pd.to_datetime(df_finale['Complete Timestamp'])

In [80]:
df_finale['Activity'] = df_finale['Activity'].str.lower()
df_finale['Activity'] = df_finale['Activity'].str.replace(' ','')

Sorting case ids by early timestamp

In [81]:
sorted_time_cases = df_finale.sort_values('Complete Timestamp')['Case ID'].drop_duplicates().values

#### Feature creation
- **Duration**: Duration of previous activity
- **Passed Time**: Passed time between first activity and current one
- **Time to conclusion**: Remaining time to process finishes

In [82]:
from tqdm import tqdm

data = []

for case in tqdm(sorted_time_cases):
    #print(case)
    df_case = df_finale[df_finale['Case ID'] == case]
    df_case['Complete Timestamp Shift'] = df_case['Complete Timestamp'].shift(1)
    
    ##datetime
    df_case['Duration'] = df_case['Complete Timestamp'] - df_case['Complete Timestamp Shift']
    
    ## colapse
    df_case = (df_case.groupby((df_case['Activity'] != df_case['Activity'].shift(1)).cumsum(), as_index=False)
               .agg(CaseID=("Case ID", "first"), Activity=("Activity", "first"), 
                    Duration=("Duration", "sum"), 
                    #Timestamp=("Complete Timestamp", "first")
                #Reward_avg=("Duration Float", "mean")
                )
              )
    
    ## day as float 
    df_case['Duration Float'] = df_case['Duration'] / pd.to_timedelta(1, unit='D')
    df_case['Duration Float'] = df_case['Duration Float'].fillna(0).round(4)
    
    time_to_conclusion = []
    total_time = df_case['Duration Float'].sum()
    
    for time in df_case['Duration Float']:
        total_time = total_time - time
        time_to_conclusion.append(total_time)
        
    df_case['Time to conclusion'] = time_to_conclusion
    df_case['Time to conclusion'] = df_case['Time to conclusion'].round(4)
    
    df_case['Passed Time'] = df_case['Time to conclusion'].iloc[::-1].values
    df_case['Passed Time'] = df_case['Passed Time'].round(4)
    
    df_case['Step'] = np.arange(1, df_case.shape[0]+1)    
    
    prefix = []
    for i in range(1, len(df_case['Activity'])+1):
        prefix.append(','.join(df_case['Activity'][:i].values))
    
    df_case['Prefix'] = prefix
    
    data.append(df_case)

100%|██████████| 4580/4580 [02:41<00:00, 28.36it/s]


In [83]:
"""
(df_case.groupby((df_case['Activity'] != df_case['Activity'].shift(1)).cumsum(), as_index=False)
   .agg(Activity=("Activity", "first"), 
        Duration=("Duration", "sum") 
        #Reward_avg=("Duration Float", "mean")
       )
)
"""

'\n(df_case.groupby((df_case[\'Activity\'] != df_case[\'Activity\'].shift(1)).cumsum(), as_index=False)\n   .agg(Activity=("Activity", "first"), \n        Duration=("Duration", "sum") \n        #Reward_avg=("Duration Float", "mean")\n       )\n)\n'

Example:

In [84]:
df_case[['Step', 'Duration Float', 'Passed Time', 'Time to conclusion']]

,Step,Duration Float,Passed Time,Time to conclusion
0,1,0.0000,-0.0000,33.6809
1,2,17.9590,15.0002,15.7219
2,3,0.7217,15.7219,15.0002
3,4,15.0002,33.6809,-0.0000


In [85]:
appended_data = pd.concat(data)

In [86]:
#pd.DataFrame(appended_data).to_csv('finale_time_features_colapsed.csv', index=False)
pd.DataFrame(appended_data).to_csv('finale_time_features_colapsed_prefixed.csv', index=False)

In [87]:
df_final = pd.read_csv('finale_time_features_colapsed.csv')

In [88]:
df_final.shape

(20404, 7)

In [92]:
last_activities = []
closed_cases = []
not_closed_cases = []

for name, group in df_final.groupby('CaseID'):
    if(group.tail(1)['Activity'].values[0] == 'Closed'):
        closed_cases.append(name)
    else:
        not_closed_cases.append(name)
    last_activities.append(group.tail(1)['Activity'].values[0])    

In [93]:
from collections import Counter

Counter(last_activities)

Counter({'Closed': 4557,
         'Wait': 8,
         'VERIFIED': 1,
         'Require upgrade': 3,
         'Take in charge ticket': 1,
         'Resolve ticket': 10})

In [95]:
data_case = [
    [len(closed_cases), df_final[df_final['CaseID'].isin(closed_cases)].shape[0]],
    [len(not_closed_cases), df_final[df_final['CaseID'].isin(not_closed_cases)].shape[0]]
]

In [96]:
pd.DataFrame(data_case, columns=['# of Cases', '# Activities'])

,# of Cases,# Activities
0,4557,20307
1,23,97


In [98]:
df_final[df_final['CaseID'] == 'Case 10'][['Duration Float','Time to conclusion','Passed Time','Step']]

,Duration Float,Time to conclusion,Passed Time,Step
193,0.0000,51.9565,0.0000,1
194,36.9978,14.9587,14.9586,2
195,0.0001,14.9586,14.9587,3
196,14.9586,0.0000,51.9565,4


In [99]:
df_final[df_final['CaseID'] == 'Case 67'][['Duration Float','Time to conclusion','Passed Time','Step']]

,Duration Float,Time to conclusion,Passed Time,Step
4583,0.0000,44.0052,0.0000,1
4584,11.7339,32.2713,30.1427,2
4585,2.1286,30.1427,32.2713,3
4586,30.1427,0.0000,44.0052,4
